# Data Wrangling Project

# OpenStreetMap: Cupertino, CA, USA

## Summary

In this project I used data wrangling techniques, such as assessing data for validity, accuracy, completeness, consistency and uniformity to clean OpenStreetMap data for a part of the world that I care.

# Map Area
## Location: Cupertino, California, USA

# Why I chose this area?

I chose Cupertino, CA, USA because this is where I currently live.  It is my home and am very familiar with this city.  Cupertino is a U.S. city in Santa Clara County, California.  The city is renown as a center of innovation in Silicon Valley.  Cupertino is famous as the home of high-tech giant Apple Inc. Within the 13 square miles, this city has a population of 64,000 people.  This city is known for its excellent public schools. More than 60% of its residents aged 25 years and older hold a bachelor's degree or higher, and more than 40% were born outside of the United States. Cupertino is 42 miles south of San Francisco.  As part of this data wrangling project, I wanted to check the quality of the Cupertino map data from OpenStreetMap.org.

# Cupertino Map

In [ ]:
from IPython.display import Image
Image("https://upload.wikimedia.org/wikipedia/commons/thumb/4/45/Santa_Clara_County_California_Incorporated_and_Unincorporated_areas_Cupertino_Highlighted_0617610.svg/502px-Santa_Clara_County_California_Incorporated_and_Unincorporated_areas_Cupertino_Highlighted_0617610.svg.png", width="600")

### The following steps will be taken for this project, namely:
1.  Acquire and generate a sample data
2.  Count the types of tags available in the data
3.  Audit the data for potential problems
4.  Clean the data by fixing the problems
5.  Prepare the data to be inserted into a SQL database


# Step 1 - Acquire and generate a sample data

Cupertino data was exported from https://www.openstreet.org and I downloaded a XML OSM dataset. My dataset is about 55.3 MB (uncompressed).  I used Overpass API and downloaded a custom square area.

### https://www.openstreetmap.org/relation/2221709

### A small sample of the XML is shown below:

In [ ]:
<?xml version="1.0" encoding="UTF-8"?>
<osm version="0.6" generator="Overpass API 0.7.55.4 3079d8ea">
<note>The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.</note>
<meta osm_base="2018-09-07T00:37:02Z"/>

  <bounds minlat="37.3010000" minlon="-122.0593000" maxlat="37.3401000" maxlon="-121.9872000"/>

  <node id="302883" lat="37.3376866" lon="-122.0597495" version="8" timestamp="2018-03-02T17:25:02Z" changeset="56826727" uid="4018842" user="Stephen214">
    <tag k="highway" v="motorway_junction"/>
    <tag k="ref" v="19A"/>
    <tag k="source" v="http://www.dot.ca.gov/hq/traffops/signtech/calnexus/pdf/eightyfivenorth.pdf;http://www.dot.ca.gov/hq/traffops/signtech/calnexus/pdf/eightyfivesouth.pdf"/>
  </node>

## Downloading OSM XML file and taking a sample

In [6]:
import xml.etree.ElementTree as ET
import pprint
from collections import defaultdict
import re
import csv
import codecs
import cerberus
import schema
import sqlite3

#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "cupertino.osm"  # Replace this with your osm file
SAMPLE_FILE = "sample.osm"

k = 40 # Parameter: take every k-th top level element

## Getting the elements with tags

In [7]:
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

## Taking sample from OSM_FILE

In [8]:
with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

# Step 2 - Count the types of tags available in the data

In [9]:
"""
We are using the iterative parsing to process the map file. We are going to 
find out not only what tags are there but also how many tags.  This is to get a sense 
on how much of which data we can expect to have in the map.
The count_tags function should return a dictionary with the 
tag name as the key and the number of times this tag can be encountered in 
the map as value.

"""

def count_tags(filename):
        
    tags = {}
    for event, elem in ET.iterparse(filename):
        if elem.tag in tags.keys():
            tags[elem.tag] += 1
        else:
            tags[elem.tag] = 1
        
    return tags
def test():

    tags = count_tags('cupertino.osm')
    pprint.pprint(tags)
      

if __name__ == "__main__":
    test()

{'bounds': 1,
 'member': 8970,
 'meta': 1,
 'nd': 291147,
 'node': 260421,
 'note': 1,
 'osm': 1,
 'relation': 402,
 'tag': 117807,
 'way': 27227}


From the results above, we know that there are 10 different types of tags within this data. The most common tags within this data are 'nd', 'node', 'tag' and 'way'.  We will primarily focus on these four common tags in our auditing and cleaning process.

# Step 3 - Audit the data for potential problems

For this Data Wrangling project, I decided to audit the street names and the postal codes.  The purpose of this audit is to identify any potential problems.  Let's audit the street names first followed by the postal codes.

### i. Audit the Street Names

In [10]:
# The Cupertino OSM file and its sample 

OSM_FILE = "cupertino.osm"
# SAMPLE_FILE = "sample.osm"

# Regex to get the last word in a string of words. This is where
# the street type (eg. Street) usually is.
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

# List of expected values i.e. street names that are correct.
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square",
            "Lane", "Road", "Trail", "Parkway", "Commons", "Broadway",
            "Crescent", "Way", "Circle", "Plaza", "Terrace", "West",
            "Portofino", "Sorrento", "Volante", "Loop", "Expressway",
            "Barcelona", "Madrid","Marino","Napoli", "Palamos","East", "Paviso",
            "Seville", "Creek", "Common", "North", "South"]

# Dictionary of street types that I decided to reformat by using "mapping"
mapping = { "Ave" : "Avenue",
            "Blvd": "Boulevard"
            }

def audit_street_type(street_types, street_name):
    """ Takes in an empty dictionary of street type: street name value pairs and
        the street name and uses the regex to isolate the street type. If there
        is a match, the match object is converted to a string and if the street
        type is not in the expected list, adds the street type and street name
        to the dictionary.
    """
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
        

def is_street_name(element):
    """ Checks whether the attribute k="addr:street".
    Essentially, if the key in an element tag is for a street. Returns True if
    it is.
    """
    return (element.get('k') == "addr:street")


def audit(osmfile):
    """ Iteratively parses through each element in an XML file (in this case
        for OSM). First checks if the element is a node or way element. If
        True, then the function will iterate through each tag in the node or way
        element, and run the 'is_street_name' function on it. If True, the
        function will run the 'audit_street_type' function on it.
            Returns:
                dictionary: street type:street name value pairs.
    """
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, element in ET.iterparse(osmfile, events=("start", )):
        if element.tag == "way" or element.tag == "node":
            for tag in element.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()

    return street_types

def update_name(name, mapping):
    for key,value in mapping.iteritems():
        if key in name:
            return name.replace(key,value)
    return name   

def test():
    street_dict = audit(OSM_FILE)
    pprint.pprint(dict(street_dict))

if __name__ == '__main__':
    test()

#street_dict = audit(OSM_FILE)
#pprint.pprint(dict(street_dict))

{'Ave': set(['N Blaney Ave']), 'Blvd': set(['Stevens Creek Blvd'])}


From the audit street names above, I did encounter problems with two streets. There were only two street names which needed correction. They were "N Blaney Ave" and "Stevens Creek Blvd".  I need to correct them to "Avenue" and "Boulevard" respectively in the cleaning process. 

### ii. Audit Postal Codes

In [11]:
# The Cupertino OSM file and its sample """

OSM_FILE = "cupertino.osm"
#SAMPLE_FILE = "sample.osm"

"""Regex to get the five integer US zip code.""" 
#post_code_re = re.compile(r'\d{5}')
post_code_re = re.compile(r'^\d{5}$')


""" Checks if the key in an element tag is for a postal code. Returns True if it is."""
def is_post_code(element):
    return (element.get('k') == "addr:postcode")


"""This function adds the value to a data structure."""
def audit_post_code_type(post_code_types, post_code):
    post_code_types.append(post_code)    


""" Iteratively parses through each element in an XML file (in this case
    for OSM). First, checks if the element is a node or way element. If
    True then the function will iterate through each tag in the node or way
    element, and run the 'is_post_code' function on it. If True, the
    function will run the 'audit_post_code_type' function on it.
        Returns:
            dictionary: post code type:postcode value pairs.
"""
def audit(osmfile):
    osm_file = open(osmfile, "r")
    post_code_types = []
    for event, element in ET.iterparse(osmfile, events=("start", )):
        if element.tag == "way" or element.tag == "node":
            for tag in element.iter("tag"):
                if is_post_code(tag):
                    audit_post_code_type(post_code_types, tag.attrib['v'])
    osm_file.close()

    return post_code_types

def test():
    post_code_dict = audit(OSM_FILE)
    post_code_set = set(post_code_dict)
    pprint.pprint(post_code_set)

if __name__ == '__main__':
    test()

set(['94024',
     '95014',
     '95014-0100',
     '95014-0101',
     '95014-0106',
     '95014-0107',
     '95014-0108',
     '95014-0109',
     '95014-0110',
     '95014-0118',
     '95014-0119',
     '95014-0120',
     '95014-0121',
     '95014-0125',
     '95014-0127',
     '95014-0128',
     '95014-0141',
     '95014-0142',
     '95014-0337',
     '95014-0459',
     '95014-0505',
     '95014-0506',
     '95014-0507',
     '95014-0508',
     '95014-0513',
     '95014-0514',
     '95014-0516',
     '95014-0519',
     '95014-0520',
     '95014-0521',
     '95014-0523',
     '95014-0525',
     '95014-0526',
     '95014-0528',
     '95014-0541',
     '95014-0542',
     '95014-0544',
     '95014-0553',
     '95014-0554',
     '95014-0557',
     '95014-0562',
     '95014-0563',
     '95014-0605',
     '95014-0608',
     '95014-0614',
     '95014-0615',
     '95014-0621',
     '95014-0706',
     '95014-0708',
     '95014-0739',
     '95014-0801',
     '95014-0802',
     '95014-0803',
   

     '95014-2611',
     '95014-2612',
     '95014-2613',
     '95014-2614',
     '95014-2616',
     '95014-2617',
     '95014-2618',
     '95014-2619',
     '95014-2620',
     '95014-2621',
     '95014-2622',
     '95014-2623',
     '95014-2624',
     '95014-2625',
     '95014-2626',
     '95014-2628',
     '95014-2629',
     '95014-2630',
     '95014-2631',
     '95014-2632',
     '95014-2633',
     '95014-2634',
     '95014-2635',
     '95014-2636',
     '95014-2637',
     '95014-2638',
     '95014-2639',
     '95014-2640',
     '95014-2641',
     '95014-2642',
     '95014-2643',
     '95014-2644',
     '95014-2645',
     '95014-2646',
     '95014-2647',
     '95014-2648',
     '95014-2649',
     '95014-2650',
     '95014-2651',
     '95014-2652',
     '95014-2653',
     '95014-2654',
     '95014-2655',
     '95014-2656',
     '95014-2657',
     '95014-2658',
     '95014-2659',
     '95014-2660',
     '95014-2661',
     '95014-2662',
     '95014-2663',
     '95014-2664',
     '95014-

     '95014-3831',
     '95014-3832',
     '95014-3833',
     '95014-3834',
     '95014-3835',
     '95014-3836',
     '95014-3837',
     '95014-3838',
     '95014-3839',
     '95014-3840',
     '95014-3841',
     '95014-3842',
     '95014-3843',
     '95014-3844',
     '95014-3845',
     '95014-3846',
     '95014-3847',
     '95014-3848',
     '95014-3849',
     '95014-3850',
     '95014-3851',
     '95014-3852',
     '95014-3853',
     '95014-3854',
     '95014-3855',
     '95014-3856',
     '95014-3857',
     '95014-3858',
     '95014-3859',
     '95014-3860',
     '95014-3861',
     '95014-3862',
     '95014-3863',
     '95014-3864',
     '95014-3865',
     '95014-3866',
     '95014-3867',
     '95014-3868',
     '95014-3869',
     '95014-3870',
     '95014-3871',
     '95014-3872',
     '95014-3873',
     '95014-3874',
     '95014-3875',
     '95014-3876',
     '95014-3877',
     '95014-3878',
     '95014-3879',
     '95014-3880',
     '95014-3881',
     '95014-3882',
     '95014-

     '95014-4779',
     '95014-4780',
     '95014-4781',
     '95014-4782',
     '95014-4783',
     '95014-4784',
     '95014-4785',
     '95014-4786',
     '95014-4787',
     '95014-4788',
     '95014-4789',
     '95014-4790',
     '95014-4791',
     '95014-4792',
     '95014-4793',
     '95014-4794',
     '95014-4795',
     '95014-4800',
     '95014-4801',
     '95014-4802',
     '95014-4803',
     '95014-4804',
     '95014-4805',
     '95014-4806',
     '95014-4807',
     '95014-4808',
     '95014-4809',
     '95014-4810',
     '95014-4811',
     '95014-4812',
     '95014-4813',
     '95014-4814',
     '95014-4815',
     '95014-4816',
     '95014-4817',
     '95014-4818',
     '95014-4819',
     '95014-4820',
     '95014-4821',
     '95014-4822',
     '95014-4823',
     '95014-4825',
     '95014-4826',
     '95014-4827',
     '95014-4828',
     '95014-4829',
     '95014-4830',
     '95014-4831',
     '95014-4832',
     '95014-4833',
     '95014-4834',
     '95014-4836',
     '95014-

From the results of the postal codes above, I encountered several problems.  Majority of the postal codes were either in 5 digit postal codes or 9 digit postal codes.  I decided to standardize the postal codes to 5 digits in the cleaning process.  In addition, there is a "CA" in one of the postal codes and we need to fix this problem.  Also, one of the postal codes says "CUPERTINO" instead of the postal code digits.  We need to clean these problems.  

# Step 4 - Clean the data by fixing th problems

We have identified the problems in our audit process above.  The next step is to clean and fix the problems. I used Python functions to standardize the street names and postal codes data field.  Regular expressions were used to help with this cleaning process. 

### i. Cleaning Street Names

A mapping dictionary is created to change "Ave" and "Blvd" to "Avenue" and "Boulevard". By using the mapping dictionary, we create an 'update_name()' function to clean the street name.  The 'update_name()' function updates the old street name to the new street name.

In [12]:
# The Cupertino OSM file and its sample 

OSM_FILE = "cupertino.osm"
# SAMPLE_FILE = "sample.osm"

# Regex to get the last word in a string of words. 
# This is where the street type (eg. Street) usually is.
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

# List of expected values i.e. street names that are correct.
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square",
            "Lane", "Road", "Trail", "Parkway", "Commons", "Broadway",
            "Crescent", "Way", "Circle", "Plaza", "Terrace", "West",
            "Portofino", "Sorrento", "Volante", "Loop", "Expressway",
            "Barcelona", "Madrid","Marino","Napoli", "Palamos","East", "Paviso",
            "Seville", "Creek", "Common", "North", "South"]

# Dictionary of street types that I decided to reformat by using "mapping"
mapping = { "Ave" : "Avenue",
            "Blvd": "Boulevard"
            }

# This function will search the input street name.  If it is within the "expected" list
# then it adds the match as a key and add the string into the set.
def audit_street_type(street_types, street_name):
    """ Takes in a empty dictionary of street type: street name value pairs and
        the street name and uses the regex to isolate the street type. If there
        is a match, the match object is converted to a string and if the street
        type is not in the expected list, adds the street type and street name
        to the dictionary.
    """
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
        

def is_street_name(element):
    """ Checks whether the attribute k="addr:street".
    Essentially, if the key in an element tag is for a street. Returns True if
    it is.
    """
    return (element.get('k') == "addr:street")


def audit(osmfile):
    """ Iteratively parses through each element in an XML file (in this case
        for OSM). First checks if the element is a node or way element. If
        True, then the function will iterate through each tag in the node or way
        element, and run the 'is_street_name' function on it. If True, the
        function will run the 'audit_street_type' function on it.
        Returns:
            dictionary: street type:street name value pairs.
    """
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, element in ET.iterparse(osmfile, events=("start", )):
        if element.tag == "way" or element.tag == "node":
            for tag in element.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()

    return street_types

# The update_name function updates the old street name with the new street name
# If the street name has the defined string in the mapping dictionary then the 
# change will happen
def update_name(name, mapping):
    for key,value in mapping.iteritems():
        if key in name:
            return name.replace(key,value)
    return name   

st_types = audit(OSM_FILE)

#pprint.pprint(dict(st_types))
for st_type, ways in st_types.iteritems():
    for name in ways:
        better_name = update_name(name, mapping)
        print name, "=>", better_name        

Stevens Creek Blvd => Stevens Creek Boulevard
N Blaney Ave => N Blaney Avenue


After cleaning the street names, we can see that the "Blvd" in Stevens Creek is now "Boulevard" and "Ave" in N Blaney is now "Avenue".  

### Cleaning and Fixing the Street Names for the shape_element function

Given that 'shape_element()' is the part of this code that parses the individual elements, that is where we will use the cleaning function(s) from the auditing part of the project.

In [13]:
#Regular expressions used to aid data cleaning section
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


# List of expected values i.e. street names that are correct.
# Street types that does not require cleaning.
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square",
            "Lane", "Road", "Trail", "Parkway", "Commons", "Broadway",
            "Crescent", "Way", "Circle", "Plaza", "Terrace", "West",
            "Portofino", "Sorrento", "Volante", "Loop", "Expressway",
            "Barcelona", "Madrid","Marino","Napoli", "Palamos","East", "Paviso",
            "Seville", "Creek", "Common", "North", "South"]

# Dictionary of street types that needs reformatting
mapping = { "Ave" : "Avenue",
            "Blvd": "Boulevard"
            }       
       
# The two functions below will be used in the def shape_element function to format the street name.

# Given that the attributes that we want to clean will typically be the 'tag' elements 
# for 'node' and 'way' parent elements, then the  section of the code that we will use the 
# cleaning function will be for those 'tag' elements.


def update_street_name(name, mapping):
    for key,value in mapping.iteritems():
        if key in name:
            return name.replace(key,value)
    return name       


def audit_street_name_tag(element): 
    street_name=element.get('v')
    m = street_type_re.search(street_name)
    if m:
        better_street_name=update_street_name(street_name,mapping)
        return better_street_name
    return street_name




### ii. Cleaning Postal Codes

From the audit, I noticed that the postal codes were split between 5 digits and 9 digits.  I decided to standardize all postal codes to 5 digits.  I also decided to drop "CA" from the postal codes and remove incorrect postal codes which was identified during the auditing process.

In [14]:
'''
We want all the postal codes in the standard 5 digit style. This means we have to change the following 
postal codes:-
1. Postal codes with more than 5 digits
2. Postal codes with "CA" next to it 
3. Postal codes that differ from the standard 5 digit style

'''


postcode_type_re = re.compile(r'^\d{5}$')

postcode_types = defaultdict(set)

expected_postcode = {}

def audit_post_codes(postcode_types, postcode_name, regex, expected_postcode):
    m = regex.search(postcode_name)
    if m:
        postcode_type = m.group()
        if postcode_type not in expected_postcode:
             postcode_types[postcode_type].add(postcode_name)

def is_postcode_name(elem):
    return (elem.attrib['k'] == "addr:postcode")


def audit(filename, regex):
    for event, elem in ET.iterparse(filename, events=("start",)):
        if elem.tag == "way" or elem.tag == "node":
            for tag in elem.iter("tag"):
                if is_postcode_name(tag):
                    audit_post_codes(postcode_types, tag.attrib['v'], regex, expected_postcode)
    pprint.pprint(dict(postcode_types))


'''
Using the OSM_FILE as an input to audit the postcodes
'''   
    
audit(OSM_FILE, postcode_type_re)


for postcode_type, ways in postcode_types.iteritems(): 
        for name in ways:
            if "-" in name:
                name = name.split("-")[0].strip()
            if "CA " in name:
                name = name.split("CA ")[1].strip('CA ')
            elif len(str(name))>5:
                name=name[0:5]
            elif name.isdigit()==False:
                print 'OK'
            print name

{'94024': set(['94024']),
 '95014': set(['95014']),
 '95051': set(['95051']),
 '95125': set(['95125']),
 '95129': set(['95129']),
 '95914': set(['95914'])}
95129
95125
94024
95051
95914
95014


From the cleaning results above, all postal codes have been standarized to 5 digits.  The "CA" and "CUPERTINO" has been removed from the postal codes.

### Cleaning and Fixing the Postal Codes for the shape_element function

Given that 'shape_element()' is the part of this code that parses the individual elements, that is where we will use the cleaning function(s) from the auditing part of the project.

In [15]:
# The two functions below will be used in the def shape_element function to format the postal codes

# Given that the attributes we want to clean will typically be 'tag' elements 
# for 'node' and 'way' parent elements then the section of the code that we will use the 
# cleaning function will be for those 'tag' elements

def update_postcode(name): 
    if "-" in name:
        name = name.split("-")[0].strip()
    elif "CA" in name:
        name = name.split("CA ")[1].strip('CA ')
    elif len(str(name))>5:
        name=name[0:5]
    elif name.isdigit()==False:
         name=00000
    return name



def audit_postcode_tag(element,regex=re.compile(r'^\d{5}$')):
    post_code=element.get('v')
    m = regex.search(post_code)
    if m:
        better_postcode=update_postcode(post_code)
        return better_postcode
    return post_code

# Step 5 - Prepare the data to be inserted for SQL Database

After auditing and cleaning is complete, the next step is to prepare the data to be inserted into a SQL database. I used sqlite3 to create the database. We will parse the elements in the OSM XML file, transforming them from document format to tabular format, thus making it possible to write to csv files.  These csv files can then easily be imported to a SQL database as tables.  Moreover, in the conversion process, the data was validated against a predefined schema.py file to ensure that both the structure of the csv files and the types of the data entered were as expected.  

### i. Utilize a schema

Before importing the data into a SQL databse, we convert the XML data into CSV by utilizing a schema.  The Python schema will have the same columns as the schema of the SQL tables. The schema helps to organize the XML data as we parse through each element.

In [16]:
# Note: The schema is stored in a .py file in order to take advantage of the
# int() and float() type coercion functions. Otherwise it could easily stored as
# as JSON or another serialized format.

schema = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    }
}


### ii. Defining the CSV files and their columns

In [18]:
NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema

NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

### iii. Shape each element

The 'shape_element()' function parses the information from each parent element, and its children, into dictionaries. It is these dictionaries that are returned from this function.  The dictionaries returned from the 'shape_element()' function are then processed by 'process_map()' and '.csv' files are generated from them.

The shape_element()function iteratively parses through each element.  If it is a node or way element with their respective tags then it will organize the data according to the schema.  See schema under section "Utilize a schema".

This is also a great opportunity to update the incorrect street names and postal codes onto the file.

In [19]:
"""Clean and shape node or way XML element to Python dictionary"""

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    
    
    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = [] # Handle secondary tags the same way for both node and way elements
             

   

    if element.tag=='node':
        for field in node_attr_fields:
            node_attribs[field]=element.get(field)
                 
        if element.find('tag') is None:
            pass
#           print 'No Tags'
           
        elif element.find('tag') is not None:
            tag_attrib={}
            node_tag_fields=NODE_TAGS_FIELDS
            for tag in element.iter('tag'):
                if PROBLEMCHARS.search(tag.attrib['k']):
                    pass
                elif LOWER_COLON.search(tag.attrib['k']):
                    tag_attrib[node_tag_fields[0]]=element.get('id')
                    tag_attrib[node_tag_fields[1]]=tag.get('k')[(tag.get('k').find(':')+1):]
                    if tag.attrib['k']== "addr:street":
                        tag_attrib[node_tag_fields[2]]=audit_street_name_tag(tag)
                    elif tag.attrib['k']== "addr:postcode":
                        tag_attrib[node_tag_fields[2]]=audit_postcode_tag(tag)       
                    else:
                        tag_attrib[node_tag_fields[2]]=tag.get('v')
                    tag_attrib[node_tag_fields[3]]=tag.get('k').split(':')[0]
                    tags.append(tag_attrib.copy())
                    
                else:
                    tag_attrib[node_tag_fields[0]]=element.get('id')
                    tag_attrib[node_tag_fields[1]]=tag.get('k')
                    if tag.attrib['k']== "addr:street":
                        tag_attrib[node_tag_fields[2]]=audit_street_name_tag(tag)
                    elif tag.attrib['k']== "addr:postcode":
                        tag_attrib[node_tag_fields[2]]=audit_postcode_tag(tag)    
                    else:    
                        tag_attrib[node_tag_fields[2]]=tag.get('v')
                    tag_attrib[node_tag_fields[3]]=default_tag_type
                    tags.append(tag_attrib.copy())
            
#        pprint.pprint( {'node':node_attribs,'node_tags':tags})        
                
    elif element.tag=='way':
        for field in way_attr_fields:
            way_attribs[field]=element.get(field)
    
        way_node_attrib={}
        way_node_fields=WAY_NODES_FIELDS
        for nd in element.findall('nd'):
            way_node_attrib[way_node_fields[0]]=element.get('id')
            way_node_attrib[way_node_fields[1]]=nd.get('ref')
            way_node_attrib[way_node_fields[2]]=element.findall('nd').index(nd)
            way_nodes.append(way_node_attrib.copy())
#       pprint.pprint({'way':way_attribs,'way_nodes':way_nodes})
        
           
        
        if element.find('tag') is None:
            pass
#           print 'No Tags'
           
        elif element.find('tag') is not None:
            way_tag_attrib={}
            way_tag_fields=WAY_TAGS_FIELDS
            for tag in element.iter('tag'):
                if PROBLEMCHARS.search(tag.attrib['k']):
                    pass
                elif LOWER_COLON.search(tag.attrib['k']):
                    way_tag_attrib[way_tag_fields[0]]=element.get('id')
                    way_tag_attrib[way_tag_fields[1]]=tag.get('k')[(tag.get('k').find(':')+1):]
                    if tag.attrib['k']== "addr:street":
                        way_tag_attrib[way_tag_fields[2]]=audit_street_name_tag(tag)
                    elif tag.attrib['k']== "addr:postcode":
                        way_tag_attrib[way_tag_fields[2]]=audit_postcode_tag(tag)    
                    else:
                        way_tag_attrib[way_tag_fields[2]]=tag.get('v')
                    way_tag_attrib[way_tag_fields[3]]=tag.get('k').split(':')[0]
                    tags.append(way_tag_attrib.copy())
                    
                else:
                    way_tag_attrib[way_tag_fields[0]]=element.get('id')
                    way_tag_attrib[way_tag_fields[1]]=tag.get('k')
                    if tag.attrib['k']== "addr:street":
                        way_tag_attrib[way_tag_fields[2]]=audit_street_name_tag(tag) 
                    elif tag.attrib['k']== "addr:postcode":
                        way_tag_attrib[way_tag_fields[2]]=audit_postcode_tag(tag)    
                    else:   
                        way_tag_attrib[way_tag_fields[2]]=tag.get('v')
                    way_tag_attrib[way_tag_fields[3]]=default_tag_type
                    tags.append(way_tag_attrib.copy())
#        pprint.pprint({'way':way_attribs,'way_tags':tags})
#        pprint.pprint({'way':way_attribs,'way_nodes':way_nodes,'way_tags':tags})
        
    

    
    if element.tag == 'node':
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

### iv. Helper function - Validating the element

In [20]:
def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))

### v. Helper function - UnicodeDictWriter

In [21]:
class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

### vi. Writing CSV Files

The final step is to convert the data from XML to CSV using the schema, shape_element function and the process_map function that actually writes the data into .csv files.

The process_map function uses helper functions, such as validate_element as well as class UnicodeDictWriter to make the code more organized.

In [22]:
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

### vii.  Loading the data into CSV file from OSM

In [23]:
process_map(OSM_FILE, validate=True)

## Creating and connecting to SQL Database

In [24]:
# Connect to the database
# "cupertino" is the name of the sqlite database file
db = sqlite3.connect("cupertino")  

# Get a cursor object
c = db.cursor() 

## Create nodes Table

In [25]:
# creating nodes table in database cupertino and inserting values into table nodes

query="DROP TABLE IF EXISTS nodes;" # Dropping the table if it already exists
c.execute(query);
db.commit()
query = "CREATE TABLE nodes (id INTEGER PRIMARY KEY NOT NULL,lat REAL,lon REAL,user TEXT,uid INTEGER,version INTEGER,changeset INTEGER,timestamp TEXT);"
c.execute(query)
db.commit()

# Read in the csv file as a dictionary, format the data as a list of tuples:

with open('nodes.csv','rb') as f: 
    dr = csv.DictReader(f)
    to_db = [(i['id'].decode("utf-8"),i['lat'].decode("utf-8"),i['lon'].decode("utf-8"),i['user'].decode("utf-8"),i['uid'].decode("utf-8"),i['version'].decode("utf-8"),i['changeset'].decode("utf-8"),i['timestamp'].decode("utf-8")) for i in dr]
    
# insert the formatted data

c.executemany("INSERT INTO nodes (id, lat, lon, user, uid, version, changeset, timestamp) VALUES (?,?,?,?,?,?,?,?);", to_db)
db.commit()
f.close()

## Create nodes_tag Table

In [26]:
# creating nodes_tags table in database cupertino and inserting values into table nodes_tags

query="DROP TABLE IF EXISTS nodes_tags;"  # Dropping the table if it already exists
c.execute(query);
db.commit()
query = "CREATE TABLE nodes_tags (id INTEGER,key TEXT,value TEXT,type TEXT,FOREIGN KEY (id) REFERENCES nodes(id));"
c.execute(query)
db.commit()

# Read in the csv file as a dictionary, format the data as a list of tuples:

with open('nodes_tags.csv','rb') as f: 
    dr = csv.DictReader(f)
    to_db = [(i['id'].decode("utf-8"),i['key'].decode("utf-8"),i['value'].decode("utf-8"),i['type'].decode("utf-8")) for i in dr]

# insert the formatted data

c.executemany("INSERT INTO nodes_tags (id, key, value, type) VALUES (?,?,?,?);", to_db)
db.commit()
f.close()

## Create ways Table

In [27]:
# creating ways table in database cupertino and inserting values into table ways

query="DROP TABLE IF EXISTS ways;"   # Dropping the table if it already exists
c.execute(query);
db.commit()
query = "CREATE TABLE ways(id INTEGER PRIMARY KEY NOT NULL,user TEXT,uid INTEGER,version TEXT,changeset INTEGER,timestamp TEXT);"
c.execute(query)
db.commit()

# Read in the csv file as a dictionary, format the data as a list of tuples:

with open('ways.csv','rb') as f: 
    dr = csv.DictReader(f)
    to_db = [(i['id'].decode("utf-8"),i['user'].decode("utf-8"),i['uid'].decode("utf-8"),i['version'].decode("utf-8"),i['changeset'].decode("utf-8"),i['timestamp'].decode("utf-8")) for i in dr]

# insert the formatted data        
    
c.executemany("INSERT INTO ways (id, user, uid, version, changeset, timestamp) VALUES (?,?,?,?,?,?);", to_db)
db.commit()
f.close()

## Create ways_nodes Table

In [28]:
# creating ways_nodes table in database cupertino and inserting values into table ways_nodes

query="DROP TABLE IF EXISTS ways_nodes;"  # Dropping the table if it already exists
c.execute(query);
db.commit()
query = "CREATE TABLE ways_nodes (id INTEGER NOT NULL,node_id INTEGER NOT NULL,position INTEGER NOT NULL,FOREIGN KEY (id) REFERENCES ways(id),FOREIGN KEY (node_id) REFERENCES nodes(id));"
c.execute(query)
db.commit()

# Read in the csv file as a dictionary, format the data as a list of tuples:

with open('ways_nodes.csv','rb') as f: 
    dr = csv.DictReader(f)
    to_db = [(i['id'].decode("utf-8"),i['node_id'].decode("utf-8"),i['position'].decode("utf-8")) for i in dr]

    
# insert the formatted data 
    
c.executemany("INSERT INTO ways_nodes (id, node_id, position) VALUES (?,?,?);", to_db)
db.commit()
f.close()

## Create ways_tags Table

In [29]:
# creating ways_tags table in database cupertino and inserting values into table ways_nodes

query="DROP TABLE IF EXISTS ways_tags;"   # Dropping the table if it already exists
c.execute(query);
db.commit()
query = "CREATE TABLE ways_tags (id INTEGER NOT NULL,key TEXT NOT NULL,value TEXT NOT NULL,type TEXT,FOREIGN KEY (id) REFERENCES ways(id));"
c.execute(query)
db.commit()


# Read in the csv file as a dictionary, format the data as a list of tuples:

with open('ways_tags.csv','rb') as f: 
    dr = csv.DictReader(f)
    to_db = [(i['id'].decode("utf-8"),i['key'].decode("utf-8"),i['value'].decode("utf-8"),i['type'].decode("utf-8")) for i in dr]

# insert the formatted data 

c.executemany("INSERT INTO ways_tags (id, key, value, type) VALUES (?,?,?,?);", to_db)
db.commit()
f.close()

# Data Overview

Here are the summary of the files:

OSM file .......... 55.3 MB

nodes.csv .......... 21.6 MB

nodes_tags.csv .......... 476 KB

ways.csv .......... 1.73 MB

ways_tags.csv .......... 3.43 MB

ways_nodes.csv .......... 6.98 MB


## Number of Nodes

In [30]:
query = "SELECT count(DISTINCT(id)) FROM nodes;"
c.execute(query)
rows=c.fetchall()

pprint.pprint(rows)

[(260421,)]


## Number of Ways

In [31]:
query = "SELECT count(DISTINCT(id)) FROM ways;"
c.execute(query)
rows=c.fetchall()

pprint.pprint(rows)

[(27227,)]


## Number of Unique Users

In [32]:
query = "SELECT COUNT(DISTINCT(e.uid))FROM (SELECT uid FROM Nodes UNION ALL SELECT uid FROM Ways) as e;"
c.execute(query)
rows=c.fetchall()

pprint.pprint(rows)

[(354,)]


## Top Ten Contributing Users

In [33]:
query = "select e.user, count(*) as num from (select user from nodes UNION ALL select user from ways) as e group by user order by num desc limit 10;"

c.execute(query)
rows=c.fetchall()
print 'Top 10 contributing users and their contribution:\n'
pprint.pprint(rows)

Top 10 contributing users and their contribution:

[(u'dannykath', 37269),
 (u'n76_cupertino_import', 34989),
 (u'karitotp', 31529),
 (u'calfarome', 27247),
 (u'RichRico', 24668),
 (u'andygol', 23536),
 (u'samely', 15590),
 (u'Luis36995', 13993),
 (u'Eureka gold', 10852),
 (u'doug_sfba', 7136)]


## Top 10 ammenities

In [34]:
query = "SELECT value, COUNT(*) as num FROM Nodes_tags WHERE key = 'amenity' GROUP BY value ORDER BY num desc limit 10;"
c.execute(query)
rows=c.fetchall()
print 'Top 10 amenities:\n'
pprint.pprint(rows)

Top 10 amenities:

[(u'bench', 123),
 (u'restaurant', 69),
 (u'parking_entrance', 36),
 (u'bicycle_parking', 34),
 (u'cafe', 25),
 (u'toilets', 19),
 (u'fast_food', 14),
 (u'bank', 11),
 (u'fuel', 7),
 (u'place_of_worship', 7)]


## Biggest Religion

In [35]:
query = "SELECT Nodes_tags.value, COUNT(*) as num FROM Nodes_tags JOIN (SELECT DISTINCT(id) FROM Nodes_tags WHERE value = 'place_of_worship') as sub ON Nodes_tags.id = sub.id WHERE Nodes_tags.key = 'religion' GROUP BY Nodes_tags.value ORDER BY num DESC LIMIT 1;"
c.execute(query)
rows=c.fetchall()
print 'Biggest Religion:\n'
pprint.pprint(rows)

Biggest Religion:

[(u'christian', 7)]


## Most Popular Food

In [36]:
query = "SELECT Nodes_tags.value, COUNT(*) as num FROM Nodes_tags JOIN (SELECT DISTINCT(id) FROM Nodes_tags WHERE value = 'restaurant') as sub ON Nodes_tags.id = sub.id WHERE Nodes_tags.key = 'cuisine' GROUP BY Nodes_tags.value ORDER BY num DESC LIMIT 15;"
c.execute(query)
rows=c.fetchall()
print 'Most Popular Food:\n'
pprint.pprint(rows)

Most Popular Food:

[(u'chinese', 9),
 (u'pizza', 6),
 (u'japanese', 5),
 (u'asian', 3),
 (u'thai', 3),
 (u'ice_cream', 2),
 (u'sandwich', 2),
 (u'burger', 1),
 (u'greek', 1),
 (u'indian', 1),
 (u'international', 1),
 (u'italian', 1),
 (u'korean', 1),
 (u'mexican', 1),
 (u'noodle', 1)]


# Ideas for Improvement

From the query about the "Most Popular Food", I noticed that the indian food shows only "1".  However, there are more than one indian food in the Cupertino area. Next, "asian" food is listed as one of the most popular food.  In my opinon, Asian food could be Chinese, Indian, Thai, Japanesee, etc. Hence, the question of accuracy, completeness and uniformity arises on the data quality.  

How can we encourage more users to participate in the OSM data? How frequently should the users update the OSM data to reflect the latest information on the map?  How do we educate the users on the field units so it conforms to a standard format? One thing that might address these problems is to provide incentives to the users so that they are motivated to enter the data accurately to the OSM map in a timely manner.  

The quality of the the openstreetmap is directly dependent on the users.  When there are more user participation in the openstreetmap then it is also prone to human errors. These errors impact the map data quality.  Perhaps we can have a system in place to address the validity, accuracy, completeness, consistency and uniformity of the data.  In addition, we can have bots fill map information in rural areas.

# Conclusion

Auditing and cleaning the data is a long process. It is by far the most crucial part in data analysis.  Data scientists can spend up to 50 - 80 % of their time auditng and cleaning data.  Data wrangling is an iterative procedure. Just as I had expected, I had to work through these auditing and cleaning steps several times so that the data can be easily explored and analyzed.  Unfortunately, it is difficult to identify mistakes and where these mistakes are embedded in the data.  Hence, going through the auditing and cleaning process several times help identify these errors.  As a result, we can make accurate decisions which can impact business results.

# References used:

1. Udacity Forum
2. Stackoverflow
3. Github Repositories and Websites
    a. https://gist.github.com/carlward/54ec1c91b62a5f911c42#file-sample_project-md
    b. https://gist.github.com/swwelch/f1144229848b407e0a5d13fcb7fbbd6f
    c. http://puwenning.github.io/2016/02/10/P3-project-openstreetmap-data-case-study/
    d. http://luizschiller.com/openstreetmap/
    e. https://baocongchen.github.io/Wrangle-OpenStreetMap-Data/
    f. https://github.com/pratyush19/Udacity-Data-Analyst-Nanodegree/tree/master/P3-OpenStreetMap-Wrangling-with-SQL
    g. https://github.com/dksmith01/Miami-OSM-Wrangling/blob/master/P3%20Open%20Street%20Map%20Wrangling%20%2B%20MongoDB%20queries.ipynb
    h. https://olegleyz.github.io/data_wrangling.html
    i. http://napitupulu-jon.appspot.com/posts/wrangling-openstreetmap.html
    j. https://github.com/lmarkely/openstreetmap/blob/master/OSM%20Project%20Code.ipynb
    